In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

## Get the reviews from all the existing car brands (handle csv files)

We should keep only the files (from edmunds.com) corresponding to the unique car brands from the autovit dataset.

In [6]:
# get unique brands from the autovit dataset
df = pd.read_csv("../data/autovit_data.csv")
unique_brands = [
    brand.lower().replace("-", "").replace(" ", "")
    for brand in df["make"].unique()
]

print(unique_brands)
print(len(unique_brands))

['ford', 'audi', 'toyota', 'skoda', 'bmw', 'chevrolet', 'dacia', 'mitsubishi', 'jaguar', 'mercedesbenz', 'kia', 'volkswagen', 'seat', 'honda', 'citroën', 'hyundai', 'suzuki', 'opel', 'renault', 'volvo', 'mini', 'porsche', 'peugeot', 'nissan', 'mazda', 'landrover', 'fiat', 'alfaromeo', 'jeep', 'lamborghini', 'bentley', 'lexus']
32


In [ ]:
files_path = "../data/archive/"
brands_dict = {}
pattern = r'_(.*?)\.csv'
regex = re.compile(pattern)

for csv_file in os.listdir(files_path):
    brand = csv_file.rsplit('_', 1)[-1].split('.')[0].lower()
    brand = brand.replace("-", "").replace(" ", "")
    brands_dict[brand] = csv_file 

print(len(brands_dict))
print(brands_dict)

In total, there are 50 car brands, and in the end, there should be 32, once correlated with the brands from the Autovit dataset.

In [43]:
# get file names
filenames = [
    filename
    for brand, filename in brands_dict.items()
    if brand in unique_brands
]

len(filenames)

25

It seems that in the end there are 25 car brands. So, 7 brands are missing, including Skoda, Seat, etc.

In [45]:
df_concat = pd.DataFrame()
dataframes_from_csv = []

for csv_filename in filenames:
    df_temp = pd.read_csv(files_path + csv_filename, lineterminator="\n")
    dataframes_from_csv.append(df_temp)

df_concat = pd.concat(dataframes_from_csv, ignore_index=True)
df_concat.shape

(167313, 7)

#### Save reviews in their original form before preprocessing

In [46]:
df_concat.to_csv("../data/edmunds_reviews.csv")

## Preprocessing reviews
1. remove contractions
2. remove punctuation
3. remove stopwords and lemmatize words

In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import contractions


lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words("english")
stopwords_dict = Counter(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alexandru/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/alexandru/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
# read data
df = pd.read_csv("../data/edmunds_preprocessed_reviews.csv")

# drop unnecessary columns
df.drop("Review_Date", axis=1, inplace=True)
df.drop("Unnamed: 0", axis=1, inplace=True)
df.drop("Author_Name", axis=1, inplace=True)

df.head(2)

,Vehicle_Title,Review_Title,Review,Rating
0,2007 Volkswagen New Beetle Convertible 2.5 2dr...,"New Beetle- Holds up well & Fun to Drive, but ...",i've beetle convertible 4.5 year andhave overa...,4.500
1,2007 Volkswagen New Beetle Convertible 2.5 PZE...,Quality Review,we bought car new 2007 generally satisfied. me...,4.375


### Remove punctuation and contractions (contractions first)

In [18]:
def remove_punction_and_contractions(text):
    return re.sub(r'[^\w\s]', '', contractions.fix(str(text)))

In [20]:
df["Review"] = df["Review"].apply(remove_punction_and_contractions)
df.head(2)

,Vehicle_Title,Review_Title,Review,Rating
0,2007 Volkswagen New Beetle Convertible 2.5 2dr...,"New Beetle- Holds up well & Fun to Drive, but ...",i have beetle convertible 45 year andhave over...,4.500
1,2007 Volkswagen New Beetle Convertible 2.5 PZE...,Quality Review,we bought car new 2007 generally satisfied mec...,4.375


### Remove stopwords and lemmatize

In [21]:
def remove_stopwords_and_lemmatize(text):
    text = ' '.join(
        [
            lemmatizer.lemmatize(word.lower())
            for word in text.split()
            if word.lower() not in stopwords_dict
        ]
    )
    return text

In [22]:
df["Review"] = df["Review"].apply(remove_stopwords_and_lemmatize)
df.head(2)

,Vehicle_Title,Review_Title,Review,Rating
0,2007 Volkswagen New Beetle Convertible 2.5 2dr...,"New Beetle- Holds up well & Fun to Drive, but ...",beetle convertible 45 year andhave overall hap...,4.500
1,2007 Volkswagen New Beetle Convertible 2.5 PZE...,Quality Review,bought car new 2007 generally satisfied mechan...,4.375


### Round ratings to the nearest `.5` value

In [23]:
df["Rating"] = df["Rating"].apply(lambda x: round(x * 2) / 2)
df.head(2)

,Vehicle_Title,Review_Title,Review,Rating
0,2007 Volkswagen New Beetle Convertible 2.5 2dr...,"New Beetle- Holds up well & Fun to Drive, but ...",beetle convertible 45 year andhave overall hap...,4.5
1,2007 Volkswagen New Beetle Convertible 2.5 PZE...,Quality Review,bought car new 2007 generally satisfied mechan...,4.5


### Drop rows containing `NaN` values

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167313 entries, 0 to 167312
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Vehicle_Title  167313 non-null  object 
 1   Review_Title   167294 non-null  object 
 2   Review         167313 non-null  object 
 3   Rating         167313 non-null  float64
dtypes: float64(1), object(3)
memory usage: 5.1+ MB


In [26]:
df.isna().sum()

Vehicle_Title     0
Review_Title     19
Review            0
Rating            0
dtype: int64

In [27]:
df.dropna(inplace=True)

### Save to `.csv`

In [30]:
df.to_csv("../data/final_car_reviews.csv", index=False)